In [1]:
import os
import importlib
import numpy as np
import yaml
from tqdm.auto import tqdm
import seesaw
import ray 
import seesaw.seesaw_bench
import seesaw.figures
import seesaw.dataset_manager
import seesaw.dataset

importlib.reload(seesaw.dataset)
importlib.reload(seesaw.dataset_manager)
importlib.reload(seesaw.seesaw_bench)
importlib.reload(seesaw.metrics)
importlib.reload(seesaw.figures)
importlib.reload(seesaw.seesaw_bench)

from seesaw.seesaw_bench import *
from seesaw.figures import *
from plotnine import *
from seesaw.dataset_manager import GlobalDataManager
import seesaw.figure_utils
importlib.reload(seesaw.figure_utils)
from seesaw.figure_utils import *

In [2]:
eval_data = pd.read_parquet('../figures/eval_data.parquet')

In [158]:
#eval_data.category

In [ ]:
### first data for overall eval

In [4]:
#base_cols =  ['param_hash' 'timestamp', 'dataset']
detail_cols = ['index_name', 'start_policy', 'variant', 'calibration', 'reward_horizon', 'pos_weight', 'reg_data_lambda', 'reg_query_lambda', 'reg_norm_lambda', ]

In [5]:
display_means(eval_data, other_cols=detail_cols, metric='average_precision')

dataset,index_name,start_policy,variant,calibration,reward_horizon,pos_weight,reg_data_lambda,reg_query_lambda,reg_norm_lambda,param_hash,timestamp,bdd,coco,lvis,objectnet,average
0,multiscalecoarse,after_first_batch,active_search,ground_truth,1.00,nan,nan,nan,nan,a3d1e64b,2023-03-17 18:06:03,0.72,0.93,0.66,0.65,0.74
1,multiscalecoarse,after_first_batch,active_search,ground_truth,2.00,nan,nan,nan,nan,f165d96a,2023-03-17 18:06:03,0.73,0.93,0.66,0.66,0.74
2,multiscalecoarse,after_first_batch,active_search,ground_truth,10.00,nan,nan,nan,nan,66719f9c,2023-03-17 18:06:03,0.70,0.94,0.65,0.67,0.74
3,multiscalecoarse,after_first_batch,active_search,ground_truth,60.00,nan,nan,nan,nan,ec7a1d44,2023-03-17 18:06:03,0.74,0.93,0.59,0.63,0.72
4,multiscalecoarse,after_first_batch,active_search,raw,1.00,nan,nan,nan,nan,3c2757c0,2023-03-17 18:06:03,0.70,0.90,0.64,0.62,0.72
5,multiscalecoarse,after_first_batch,active_search,raw,2.00,nan,nan,nan,nan,07c3313d,2023-03-17 18:06:03,0.69,0.89,0.64,0.62,0.71
6,multiscalecoarse,after_first_batch,active_search,raw,10.00,nan,nan,nan,nan,b9cdd550,2023-03-17 18:06:03,0.70,0.89,0.60,0.60,0.70
7,multiscalecoarse,after_first_batch,active_search,raw,60.00,nan,nan,nan,nan,386d9ea5,2023-03-17 18:06:03,0.70,0.86,0.50,0.43,0.62
8,multiscalecoarse,after_first_batch,multi_reg,nan,nan,balanced,0.00,0.00,100.00,45632884,2023-03-15 17:42:10,0.72,0.88,0.65,0.59,0.71
9,multiscalecoarse,after_first_batch,multi_reg,nan,nan,balanced,0.00,10.00,100.00,a3925409,2023-03-15 17:42:10,0.75,0.92,0.68,0.69,0.76


In [6]:
def add_hard_queries(eval_data):
    baseline_df = eval_data[eval_data.full_hash.eq('2501cb92_multiscalecoarse')]
    hard_baseline_df = baseline_df[baseline_df.average_precision < .5]
    hard_queries_df = hard_baseline_df[['dataset', 'category']].sort_values(['dataset', 'category']).reset_index(drop=True)
    eval_hard_df = eval_data.merge(hard_queries_df, left_on=('dataset', 'category'), right_on=('dataset', 'category'), how='right')

        
    eval_hard_df = eval_hard_df.assign(query_group='hard subset')
    eval_df = eval_data.assign(query_group='all queries')
    
    full_df = pd.concat([eval_df, eval_hard_df], ignore_index=True).reset_index(drop=True)
    full_df = full_df.assign(**{'query group':pd.Categorical(full_df.query_group, categories=['all queries', 'hard subset'], ordered=True)})
    
    return full_df

In [7]:
stats2 = add_hard_queries(eval_data)
# stats2.groupby(['query_group', 'dataset', 'full_hash']).size().unstack('dataset')
dataset_names = {
    'lvis':'LVIS',
    'objectnet':'ObjNet',
    'coco':'COCO',
    'bdd':'BDD',
    'Avg.':'Avg.', # used for ordering. 
}
stats2 = stats2.assign(dataset=pd.Categorical(stats2.dataset.map(lambda x : dataset_names[x]), categories=dataset_names.values(), ordered=True))

In [102]:
def basic_data(stats2):
    names = {'2501cb92_multiscalecoarse':'zero-shot CLIP',
             'eda85ef1_multiscalemed': 'this work'
            }
    
    stats2 = stats2[stats2.full_hash.isin(names.keys())]
    stats2 = stats2.assign(method=pd.Categorical(stats2.full_hash.map(lambda x : names[x]), 
                                                 categories=names.values(), ordered=True))
    
    return stats2

In [103]:
bdata = basic_data(stats2)

In [19]:
#bdata = bdata[bdata.query_group == 'all queries']

In [106]:
bdata

,param_hash,index_name,timestamp,dataset,category,full_hash,start_policy,variant,calibration,reward_horizon,pos_weight,reg_data_lambda,reg_query_lambda,reg_norm_lambda,reciprocal_rank,ndcg_score,average_precision,query_group,query group,method
2,eda85ef1,multiscalemed,2023-03-12 19:12:19,LVIS,puffer (fish),eda85ef1_multiscalemed,after_first_batch,multi_reg,None,NaN,balanced,1000.0,10.0,100.0,0.000000,0.000000,0.000000,all queries,all queries,this work
4,eda85ef1,multiscalemed,2023-03-12 19:12:19,LVIS,sword,eda85ef1_multiscalemed,after_first_batch,multi_reg,None,NaN,balanced,1000.0,10.0,100.0,1.000000,0.726581,0.700484,all queries,all queries,this work
5,eda85ef1,multiscalemed,2023-03-12 19:12:19,LVIS,rag doll,eda85ef1_multiscalemed,after_first_batch,multi_reg,None,NaN,balanced,1000.0,10.0,100.0,1.000000,1.000000,1.000000,all queries,all queries,this work
6,eda85ef1,multiscalemed,2023-03-12 19:12:19,LVIS,limousine,eda85ef1_multiscalemed,after_first_batch,multi_reg,None,NaN,balanced,1000.0,10.0,100.0,1.000000,0.710040,0.435329,all queries,all queries,this work
8,eda85ef1,multiscalemed,2023-03-12 19:12:19,ObjNet,blanket,eda85ef1_multiscalemed,after_first_batch,multi_reg,None,NaN,balanced,1000.0,10.0,100.0,0.500000,0.312505,0.491624,all queries,all queries,this work
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39083,2501cb92,multiscalecoarse,2023-03-15 17:42:10,ObjNet,water filter,2501cb92_multiscalecoarse,from_start,baseline,None,NaN,None,NaN,NaN,NaN,0.100000,0.033339,0.016250,hard subset,hard subset,zero-shot CLIP
39096,eda85ef1,multiscalemed,2023-03-12 19:12:19,ObjNet,webcam,eda85ef1_multiscalemed,after_first_batch,multi_reg,None,NaN,balanced,1000.0,10.0,100.0,0.500000,0.346538,0.405657,hard subset,hard subset,this work
39105,2501cb92,multiscalecoarse,2023-03-15 17:42:10,ObjNet,webcam,2501cb92_multiscalecoarse,from_start,baseline,None,NaN,None,NaN,NaN,NaN,0.142857,0.126992,0.139933,hard subset,hard subset,zero-shot CLIP
39117,eda85ef1,multiscalemed,2023-03-12 19:12:19,ObjNet,weight (exercise),eda85ef1_multiscalemed,after_first_batch,multi_reg,None,NaN,balanced,1000.0,10.0,100.0,0.142857,0.458784,0.441563,hard subset,hard subset,this work


In [60]:
bdata.to_parquet('../figures/basic_scatter_data.parquet')

In [54]:
gplot = (ggplot(data=bdata[bdata.query_group == 'all queries']) 
# + geom_boxplot(aes(x='dataset', y='average_precision'))
 + geom_violin(aes(x='dataset', y='average_precision'), scale='width', size=.1)
 + geom_errorbar(aes(x='dataset', ymin='average_precision', ymax='average_precision'), width=.05, size=.5, alpha=.3)
 #+ geom_rug(aes(x='average_precision'))
 #+ geom_density(aes(x='average_precision'), )
# + facet_wrap(facets=['dataset'], nrow=4, ncol=1, scales='free_y')
 + annotate('rect', xmin=-math.inf, ymin=0, xmax=math.inf, ymax=.5, fill='palegreen', alpha=.1)      
 + coord_flip()
)

In [56]:
#gplot.data

In [61]:
# (ggplot(data=bdata[bdata.query_group == 'all queries']) 
# # + geom_boxplot(aes(x='dataset', y='average_precision'))
# # + geom_violin(aes(x='dataset', y='average_precision'), scale='width')
#  + geom_rug(aes(x='average_precision'))
#  + geom_density(aes(x='average_precision'), )
#  + facet_wrap(facets=['dataset'], nrow=4, ncol=1, scales='free_y')
# # + coord_flip()
# )

In [387]:
## goal: decide what results tables will look like. 

## option 

### option 1: skip multiscale altogether.
# benefit: avoids need to explain multiscale. easy to compare with other existing approaches.
# shortcoming: thins out contribution list. user study and system description make less sense in this case.

### option 2: include multiscale only for some evaluation sections. 
## start with an overall section that compares all techniques. 
## can have with and without multiscale to include ENS, but also to introduce the results.

### Problem 1: absolute aggregate improvements can seem small. this needs to be addressed early.
# we need to show them for less common queries
## by dataset, by query hardness, by method is a lot of data.
## why is by dataset needed? to show generality.

### approach
# 1. results compared to baseline as before, but introduce good results before. this can be done with relative improvement.

# 2. now compare against ENS and Log Reg. without necessarily doing that breakdown, as we have established the small numbers are ok.

## 2.2 break-down ENS performance partly to explain results.

# 3. now break down seesaw optimization benefits. no need to break them down unless some stuff shows no benefit in one subset.

# 4. user study.



In [62]:
# stats2.groupby(['variant', 'calibration', 'reward_horizon', 'full_hash'], dropna=False).size()

In [73]:
def gen_main_comparison_table(stats2, metric='average_precision'):
    """compare the high level strategies without a benefit breakdown"""
    
    table_variant_names = {
                '2501cb92_multiscalecoarse':'CLIP alone',
        #  active search no calibration. reward horizon 60
                '386d9ea5_multiscalecoarse':'ENS', 
                'eda85ef1_multiscalecoarse':'this work$^-$',
#                'eda85ef1_multiscalemed': 'this work',
    }
    
    breakdown_df = stats2[stats2.full_hash.isin(table_variant_names.keys())] #[compare_subset.easy_full_variant_name.isin(breakdown_variants)]
    
    # print(breakdown_df.full_hash.unique())
    breakdown_df = breakdown_df.assign(method = breakdown_df.full_hash.map(lambda x : table_variant_names[x]))
    breakdown_df = breakdown_df.assign(method = pd.Categorical(breakdown_df.method, categories=list(table_variant_names.values()), ordered=True))

    breakdown_df = breakdown_df[['query group','method', 'category', 'param_hash', 'index_name', 'dataset', metric]]    
    aggs = breakdown_df.groupby(['query group', 'method','dataset'], dropna=True)[metric].mean().rename('average_precision')
    
    avg = aggs.reset_index().assign(dataset='Avg.').groupby(['query group', 'method', 'dataset'], dropna=True)[metric].mean()
    
    aggs = pd.concat([aggs, avg])
    aggs = aggs.sort_index().dropna()
    
    return aggs

In [446]:
#stats2[stats2.full_hash.isin(['2501cb92_multiscalecoarse'])]

In [70]:
gen_main_comparison_table(stats2)

query group  method       dataset
all queries  CLIP alone   LVIS       0.625204
                          ObjNet     0.635007
                          COCO       0.896128
                          BDD        0.741339
                          Avg.       0.724420
             ENS          LVIS       0.501479
                          ObjNet     0.429477
                          COCO       0.862546
                          BDD        0.696917
                          Avg.       0.622605
             this work^-  LVIS       0.685640
                          ObjNet     0.701285
                          COCO       0.923996
                          BDD        0.758466
                          Avg.       0.767347
hard subset  CLIP alone   LVIS       0.187418
                          ObjNet     0.277259
                          COCO       0.271905
                          BDD        0.021126
                          Avg.       0.189427
             ENS          LVIS       0.155569


In [94]:
tab = gen_main_comparison_table(stats2).unstack('dataset')
tab.index=tab.index.rename([None,None])
tab.columns=tab.columns.rename(None)

with pd.option_context('display.float_format', '{:.02f}'.format):
    display(tab)

LVIS  ObjNet  COCO  BDD  Avg.
all queries CLIP alone     0.63    0.64  0.90 0.74  0.72
            ENS            0.50    0.43  0.86 0.70  0.62
            this work$^-$  0.69    0.70  0.92 0.76  0.77
hard subset CLIP alone     0.19    0.28  0.27 0.02  0.19
            ENS            0.16    0.24  0.37 0.03  0.20
            this work$^-$  0.30    0.40  0.55 0.07  0.33

In [95]:
print(tab.style.format(precision=2).to_latex(label='tab:main',hrules=True, clines='all;index', caption='Table comparing \sys/ to baselines'))

\begin{table}
\caption{Table comparing \sys/ to baselines}
\label{tab:main}
\begin{tabular}{llrrrrr}
\toprule
 &  & LVIS & ObjNet & COCO & BDD & Avg. \\
\midrule
\multirow[c]{3}{*}{all queries} & CLIP alone & 0.63 & 0.64 & 0.90 & 0.74 & 0.72 \\
\cline{2-2}
 & ENS & 0.50 & 0.43 & 0.86 & 0.70 & 0.62 \\
\cline{2-2}
 & this work$^-$ & 0.69 & 0.70 & 0.92 & 0.76 & 0.77 \\
\cline{1-2} \cline{2-2}
\multirow[c]{3}{*}{hard subset} & CLIP alone & 0.19 & 0.28 & 0.27 & 0.02 & 0.19 \\
\cline{2-2}
 & ENS & 0.16 & 0.24 & 0.37 & 0.03 & 0.20 \\
\cline{2-2}
 & this work$^-$ & 0.30 & 0.40 & 0.55 & 0.07 & 0.33 \\
\cline{1-2} \cline{2-2}
\bottomrule
\end{tabular}
\end{table}



In [80]:
def gen_seesaw_breakdown_table(stats2):
        
    # baseline                        2501cb92_multiscalecoarse    1608
    #                                 2501cb92_multiscalemed       1608
    # lr_norm_and_data_reg            0ddf1e42_multiscalecoarse    1608
    #                                 0ddf1e42_multiscalemed       1608
    # lr_norm_and_query_and_data_reg  eda85ef1_multiscalecoarse    1608
    #                                 eda85ef1_multiscalemed       1608
    # lr_norm_and_query_reg           2c32d711_multiscalemed       1608
    #                                 a3925409_multiscalecoarse    1608
    # lr_norm_reg                     45632884_multiscalecoarse    1608
    #                                 56d7a279_multiscalemed       1608

#    easy_name_map = {k:v for (v,k) in easy_names}
    
    
    # subset = stats2[stats2.param_hash.isin(easy_name_map.keys())]
    # subset = subset.assign(easy_variant_name=subset.param_hash.map(lambda phash : easy_name_map[phash]))
    # subset = (subset.assign(easy_full_variant_name=compare_subset[['easy_variant_name', 'index_name']]
    #                        .apply(tuple, axis=1).map(lambda tup: f'{tup[1]}_{tup[0]}')))

    # previous version.
    # table_variant_names = {
    #                       '2501cb92_multiscalecoarse':'CLIP alone', 
    #                       'a3925409_multiscalecoarse': '+ CLIP alignment objective', 
    #                       'eda85ef1_multiscalecoarse': '+ DB alignment objective',
    #                       'eda85ef1_multiscalemed': '+ multiscale representation',
    # }
    
    table_variant_names = { 
                            '2501cb92_multiscalecoarse':'zero-shot CLIP', 
                            '2501cb92_multiscalemed':'+ multiscale rep.',
                            '56d7a279_multiscalemed':'+ few-shot LR over CLIP',
                            '2c32d711_multiscalemed':'+ CLIP alignment',
                            'eda85ef1_multiscalemed':'+ DB alignment',
    }
    
    # alternative ordering
    # table_variant_names = {
    #                       'multiscalecoarse_baseline':'CLIP alone', 
    #                       'multiscalemed_baseline': '+ multi-scale representation', 
    #                       'multiscalemed_lr_norm_and_query_reg': '+ CLIP alignment objective', 
    #                       'multiscalemed_lr_norm_and_query_and_data_reg': '+ DB alignment objective',
    # }    
    breakdown_df = stats2[stats2.full_hash.isin(table_variant_names.keys())]
    breakdown_df = breakdown_df.assign(method = breakdown_df.full_hash.map(lambda x : table_variant_names[x]))
    breakdown_df = breakdown_df.assign(method=pd.Categorical(breakdown_df.method, categories=table_variant_names.values(), ordered=True))
    
    agg = breakdown_df.groupby(['query group', 'method', 'dataset'], observed=True).average_precision.mean()
    
    avg = agg.reset_index().assign(dataset='Avg.').groupby(['query group', 'method', 'dataset'], observed=True).average_precision.mean()
    
    agg = pd.concat([agg, avg]).sort_index()
    
    return agg

In [98]:
tab = gen_seesaw_breakdown_table(stats2).unstack('dataset')
tab.index=tab.index.rename([None,None])
tab.columns=tab.columns.rename(None)

with pd.option_context('display.float_format', '{:.02f}'.format):
    display(tab)

LVIS  ObjNet  COCO  BDD  Avg.
all queries zero-shot CLIP           0.63    0.64  0.90 0.74  0.72
            + multiscale rep.        0.70    0.64  0.95 0.76  0.76
            + few-shot LR over CLIP  0.67    0.59  0.87 0.68  0.70
            + CLIP alignment         0.75    0.69  0.96 0.77  0.79
            + DB alignment           0.76    0.70  0.96 0.79  0.80
hard subset zero-shot CLIP           0.19    0.28  0.27 0.02  0.19
            + multiscale rep.        0.32    0.28  0.58 0.10  0.32
            + few-shot LR over CLIP  0.34    0.28  0.57 0.07  0.31
            + CLIP alignment         0.42    0.39  0.74 0.20  0.44
            + DB alignment           0.44    0.40  0.75 0.24  0.46

In [100]:
print(tab.style.format(precision=2).to_latex(label='tab:breakdown',hrules=True, clines='all;index', caption='breakdown of improvements'))

\begin{table}
\caption{breakdown of improvements}
\label{tab:breakdown}
\begin{tabular}{llrrrrr}
\toprule
 &  & LVIS & ObjNet & COCO & BDD & Avg. \\
\midrule
\multirow[c]{5}{*}{all queries} & zero-shot CLIP & 0.63 & 0.64 & 0.90 & 0.74 & 0.72 \\
\cline{2-2}
 & + multiscale rep. & 0.70 & 0.64 & 0.95 & 0.76 & 0.76 \\
\cline{2-2}
 & + few-shot LR over CLIP & 0.67 & 0.59 & 0.87 & 0.68 & 0.70 \\
\cline{2-2}
 & + CLIP alignment & 0.75 & 0.69 & 0.96 & 0.77 & 0.79 \\
\cline{2-2}
 & + DB alignment & 0.76 & 0.70 & 0.96 & 0.79 & 0.80 \\
\cline{1-2} \cline{2-2}
\multirow[c]{5}{*}{hard subset} & zero-shot CLIP & 0.19 & 0.28 & 0.27 & 0.02 & 0.19 \\
\cline{2-2}
 & + multiscale rep. & 0.32 & 0.28 & 0.58 & 0.10 & 0.32 \\
\cline{2-2}
 & + few-shot LR over CLIP & 0.34 & 0.28 & 0.57 & 0.07 & 0.31 \\
\cline{2-2}
 & + CLIP alignment & 0.42 & 0.39 & 0.74 & 0.20 & 0.44 \\
\cline{2-2}
 & + DB alignment & 0.44 & 0.40 & 0.75 & 0.24 & 0.46 \\
\cline{1-2} \cline{2-2}
\bottomrule
\end{tabular}
\end{table}



In [555]:
#tab_breakdown.reset_index().to_parquet('../figures/breakdown_df.parquet')

In [556]:
# print(tab_breakdown.unstack('dataset').style.format(precision=2)
#         .to_latex(label='tab:breakdown', hrules=True, clines='all;index', caption='Table breaking down contributions'))

In [463]:
# def breakdown_ens_error(stats2):
#     ''' explain what happens to ENS'''
#     pass

In [583]:

def ens_table(stats2):

    calib_map = {
            'raw':'raw CLIP score',
            'ground_truth':'otimal calibration (theoretical)',
    }
    ens_data = stats2[stats2.variant == 'active_search']
    ens_data = ens_data.assign(calibration=ens_data.calibration.map(lambda x : calib_map[x]))
    ens_data = ens_data.assign(calibration=pd.Categorical(ens_data.calibration, categories=calib_map.values(), ordered=True))
    ens_data = ens_data.assign(horizon=ens_data.reward_horizon.astype('int'))
    means = ens_data.groupby(['calibration', 'horizon', 'dataset'], observed=True).average_precision.mean()
    mri = means.reset_index()

    summary = mri.groupby(['calibration', 'horizon']).average_precision.mean()
    return summary

In [598]:
print(ens_table(stats2).unstack('horizon').style.format(precision=2).to_latex(label='tab:ens', hrules=True, caption='ens table', ))

\begin{table}
\caption{ens table}
\label{tab:ens}
\begin{tabular}{lrrrr}
\toprule
horizon & 1 & 2 & 10 & 60 \\
calibration &  &  &  &  \\
\midrule
raw CLIP score & 0.63 & 0.62 & 0.61 & 0.55 \\
otimal calibration (theoretical) & 0.65 & 0.65 & 0.65 & 0.63 \\
\bottomrule
\end{tabular}
\end{table}



In [92]:
breakdown_df.to_parquet('../figures/breakdown_df.parquet')

In [61]:
reduced_stats = stats[stats.full_hash.isin(variant_map.values())]
reduced_stats = reduced_stats.assign(variant_name=reduced_stats.full_hash.map(lambda x : variant_map_inv[x]))

In [64]:
method_comp = reduced_stats.pivot(index=['dataset', 'category'], columns=['variant_name'], values='average_precision',).reset_index()

In [67]:
method_comp.to_parquet('../seesaw_notebooks/main_results_scatter.parquet')

In [48]:
#mc = method_comp[method_comp.index_name == 'multiscalemed']

In [110]:
bdata['query group']

2        all queries
4        all queries
5        all queries
6        all queries
8        all queries
            ...     
39083    hard subset
39096    hard subset
39105    hard subset
39117    hard subset
39121    hard subset
Name: query group, Length: 4348, dtype: category
Categories (2, object): ['all queries' < 'hard subset']

In [113]:
bdatar = bdata[['dataset', 'category',  'query group', 'method', 'average_precision']]

In [118]:
#bdatar.groupby(['dataset', 'method', 'query group']).size()


dataset  method          query group
LVIS     zero-shot CLIP  all queries    1203
                         hard subset     456
         this work       all queries    1203
                         hard subset     456
ObjNet   zero-shot CLIP  all queries     313
                         hard subset     102
         this work       all queries     313
                         hard subset     102
COCO     zero-shot CLIP  all queries      80
                         hard subset       5
         this work       all queries      80
                         hard subset       5
BDD      zero-shot CLIP  all queries      12
                         hard subset       3
         this work       all queries      12
                         hard subset       3
Avg.     zero-shot CLIP  all queries       0
                         hard subset       0
         this work       all queries       0
                         hard subset       0
dtype: int64

In [124]:
pivoted = bdatar.pivot(index=['query group', 'dataset', 'category'], columns='method', values='average_precision')

In [135]:
bdata.average_precision.quantile(q=[0., .25, .5, .75, 1.])

0.00    0.000000
0.25    0.273000
0.50    0.660354
0.75    1.000000
1.00    1.000000
Name: average_precision, dtype: float64

In [142]:
medians = bdata.groupby(['query group', 'dataset', 'method']).average_precision.quantile([0., .25, .5, .75, 1.])


In [146]:
sbs = bdata.pivot(index=['query group', 'dataset', 'category'], values='average_precision', columns='method')

In [148]:
sbs = sbs.assign(delta=sbs['this work'] - sbs['zero-shot CLIP'])

In [152]:
plottable = sbs.reset_index()

In [153]:
plottable.to_parquet('../figures/seesaw_delta.parquet')

In [143]:
medians

query group  dataset  method              
all queries  LVIS     zero-shot CLIP  0.00    0.000000
                                      0.25    0.263138
                                      0.50    0.763324
                                      0.75    1.000000
                                      1.00    1.000000
                                                ...   
hard subset  Avg.     this work       0.00         NaN
                                      0.25         NaN
                                      0.50         NaN
                                      0.75         NaN
                                      1.00         NaN
Name: average_precision, Length: 100, dtype: float64

In [588]:
scatter, deltahist = plot_compare2(bdata, variant_col=variant_map['pseudo_lr'], baseline_col=variant_map['baseline'])

In [589]:
#scatter.data.to_parquet('/state/partition1/user/omoll/seesaw/figures/query_scatter_multiscale.parquet')